## RAG example with Langchain, Milvus, and OLLAMA
Requirements:
- A Milvus instance, either standalone or cluster.
- Connection credentials to Milvus must be available as environment variables: MILVUS_USERNAME and MILVUS_PASSWORD.
- An OLLAMA inference endpoint.

### Needed packages and imports

In [ ]:
!pip install -q einops==0.7.0 langchain==0.1.9 pymilvus==2.3.6 sentence-transformers==2.4.0 openai==1.13.3

In [ ]:
import os
from langchain.callbacks.base import BaseCallbackHandler
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Milvus

#### Bases parameters, Inference server and Milvus info

In [ ]:
os.environ["OPENAI_API_KEY"] = "EMPTY"
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [ ]:
# Replace values according to your Milvus deployment
### ocp route
INFERENCE_SERVER_URL ="http://ollama:11434"
MODEL_NAME = "mistral"
MILVUS_HOST = "vectordb-milvus"
MILVUS_PORT = 19530
MILVUS_USERNAME = "root"
MILVUS_PASSWORD = "Milvus"
MILVUS_COLLECTION = "redhat_notes"

#### Initialize the connection

In [ ]:
model_kwargs = {'trust_remote_code': True}
embeddings = HuggingFaceEmbeddings(
    model_name="nomic-ai/nomic-embed-text-v1",
    model_kwargs=model_kwargs,
    show_progress=False
)

store = Milvus(
    embedding_function=embeddings,
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT, "user": MILVUS_USERNAME, "password": MILVUS_PASSWORD},
    collection_name=MILVUS_COLLECTION,
    metadata_field="metadata",
    text_field="page_content",
    drop_old=False
    )

#### Initialize query chain

In [ ]:
template="""<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant named SnemeisBot answering questions.
You will be given a question you need to answer, and a context about everything a sales team wrote down about its customers to provide you with information. You must answer the question based as much as possible on this context.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, don't share false information.
<</SYS>>

Context: 
{context}

Question: {question} [/INST]
"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

llm = Ollama(base_url=INFERENCE_SERVER_URL, model=MODEL_NAME)

qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 1}
            ),
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT, "verbose": True},
        return_source_documents=True
        )

os.environ["TOKENIZERS_PARALLELISM"] = "false"

#### Query example

In [ ]:
question = "Who is your mother?"
result = qa_chain.invoke({"query": question})

#### Retrieve source

In [ ]:
type(result)

In [ ]:
result

In [ ]:
def remove_duplicates(input_list):
    unique_list = []
    for item in input_list:
        if item.metadata['source'] not in unique_list:
            unique_list.append(item.metadata['source'])
    return unique_list

results = remove_duplicates(result['source_documents'])

for s in results:
    print(s)